In [22]:
import json
import time
import speech_recognition as sr
from gtts import gTTS
import subprocess
from pydub import AudioSegment
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")
pAudio = base.audio

text=''

#conversion fichier .wav au bon format
def wavgoodform(namesourcefile):
    #conversion du fichier de départ en mono vers un fichier final en stéréo
    sound = AudioSegment.from_wav(namesourcefile)
    sound = sound.set_channels(2)
    sound.export("Voix001wavgoodversion.wav",format="wav")
    #conversion du fichier de départ en 16 bits vers un fichier en 24 bits
    data, samplerate = soundfile.read('Voix001wavgoodversion.wav')
    soundfile.write('f.wav', data, 48000, subtype='PCM_24')
    pAudio.load('f.wav')
    pAudio.play()
    
    
#enregistrement microphone
def micropynq():
    validation=0
    print("dire quelque chose")
    pAudio.record(4)
    audiotest = pAudio.save("recording_1.wav")
    r = sr.Recognizer()
    audiotest = sr.AudioFile("recording_1.wav")
    with audiotest as source: 
        audio = r.record(source)
    try:
        text=r.recognize_google(audio,language='fr-FR')
    except sr.UnknownValueError:
        validation=1
    while validation==0:
        pAudio.record(4)
        audiotest = pAudio.save("recording_1.wav")
        audiotest = sr.AudioFile("recording_1.wav")
        with audiotest as source: 
            audio = r.record(source)
        try:
            texttemp=r.recognize_google(audio,language='fr-FR')
            text=text+texttemp
        except sr.UnknownValueError:
            validation=1
            
#application générale            
r = sr.Recognizer()
#à partir d'un fichier ".wav"
audiotest = sr.AudioFile("/Users/camil/TESTPROJETMONS/heureux.wav")
with audiotest as source: 
    audiotext = r.record(source)
#à partir du microphone
micropynq()
time.sleep(3)
translator = Translator()
translations = translator.translate(text, dest='en')
print(translations)
texttrad = translations.text
texttradaudio = gTTS(text=texttrad, lang='en', slow=False)
texttradaudio.save("testvf.mp3")
subprocess.call(['ffmpeg', '-i', "testvf.mp3","testvffin.wav"])
wavegoodform("testvffin.wav")
